In [ ]:
from dotenv import load_dotenv

load_dotenv
import os

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gpt-4.1-mini",
    temperature=0.2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [ ]:
result = llm.invoke("What is an agent?")

In [ ]:
print(result)

In [ ]:
from langchain.tools import tool


@tool
def write_email(
    to: str,
    subject: str,
    content: str,
) -> str:
    """write and send an email"""
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

In [ ]:
type(write_email)

In [ ]:
write_email.args

In [ ]:
model_with_tool = llm.bind_tools(
    [write_email],
    tool_choice="any",
    parallel_tool_calls=False,
)
output = model_with_tool.invoke(
    "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"
)

In [ ]:
type(output)

In [ ]:
output

In [ ]:
args = output.tool_calls[0]["args"]
args

In [ ]:
result = write_email.invoke(args)

In [ ]:
result

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END


class state_schema(TypedDict):
    request: str
    email: str


workflow = StateGraph(state_schema)

In [ ]:
def write_email_node(state: state_schema) -> state_schema:
    output = model_with_tool.invoke(state["request"])
    args = output.tool_calls[0]["args"]
    email = write_email.invoke(args)
    return {"email": email}

In [ ]:
workflow = StateGraph(state_schema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

In [ ]:
app

In [ ]:
app.invoke(
    {
        "request": "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"
    }
)

In [ ]:
from typing import Literal
from langgraph.graph import MessagesState


def call_llm(state: MessagesState) -> MessagesState:
    """Run llm"""
    output = model_with_tool.invoke(state["messages"])
    return {"messages": [output]}


def run_tool(state: MessagesState):
    """perform the tool call"""
    result = []
    for tool_call in state["messages"][-1].tool_calls:
        observation = write_email.invoke(tool_call["args"])
        result.append(
            {"role": "tool", "content": observation, "tool_call_id": tool_call["id"]}
        )
    return {"messages": result}


def should_continue(state: MessagesState) -> Literal["run_tool", "__end__"]:
    """Route to tool handler, or end if Done tool called"""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "run_tool"
    return END


workflow = StateGraph(MessagesState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("run_tool", run_tool)
workflow.add_edge(START, "call_llm")
workflow.add_conditional_edges(
    "call_llm", should_continue, {"run_tool": "run_tool", END: END}
)
workflow.add_edge("run_tool", END)
app=workflow.compile()

In [ ]:
app

In [ ]:
result = app.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!",
            }
        ]
    }
)
for m in result["messages"]:
    m.pretty_print()

In [ ]:
from langchain.agents import create_agent

from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided.",
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!",
            }
        ]
    }
)

for m in result["messages"]:
    m.pretty_print()

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agent= create_react_agent(
    model=llm,
    tool=[write_email],
    wer
)